# Combining redis and elasticsearch

In [63]:
import redis

In [64]:
r = redis.Redis(host="192.168.10.15", port=6379)

In [65]:
key = "test-1"
r.delete(key)

1

In [66]:
msgs = ["message {}".format(i) for i in range(10)]
for msg in msgs:
    r.rpush(key, msg)

In [67]:
from elasticsearch import Elasticsearch
e = Elasticsearch(host="192.168.10.15")
e.ping()

True

In [68]:
import json
bulk = []
for msg in r.lrange(key, 0, -1):
    bulk.append({
        "index": {
            "_index": key,
            "_type": "doc"
        }
    })
    msg = msg.decode("utf-8")
    bulk.append({
        "message": msg
    })

resp = e.bulk(body=bulk)
if resp["errors"]:
    print(resp)

In [69]:
docs = e.search(key)
docs = docs["hits"]["hits"]

In [70]:
for doc in docs:
    print(doc["_source"])

{'message': 'message 1'}
{'message': 'message 5'}
{'message': 'message 6'}
{'message': 'message 8'}
{'message': 'message 7'}
{'message': 'message 1'}
{'message': 'message 3'}
{'message': 'message 5'}
{'message': 'message 6'}
{'message': 'message 9'}


# Task

* Set up an elasticsearch cluster per group;
 * Each student should run Suricata and at least one Elastic node;
 * Elastic node roles are up to you, but please distribute - in other words, someone is proxy, someone is master, someone is worker;
 * Don't install all roles on all nodes!
* Enhance the scripts written in *001-redis* notebook to function as elasticsearch shippers;
 * Each student should write their own script and should send to their personal index in the group elasticsearch;
 * You should also set up a common index where you will send messages from all students, it's up to you if you want to write a single consumer for that or if each student sends on their own;